In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
# Load training data set
train = pd.read_json('../input/train.json')

In [3]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [4]:
order_num = np.random.rand(train.shape[0])
train['order'] = order_num
train.sort_values('order', axis=0, inplace=True)
print(train.head())

                                                 band_1  \
797   [-25.211447, -23.585413, -21.033905, -20.13462...   
1440  [-31.97308, -29.474361, -29.050629, -29.050684...   
728   [-23.622217, -21.182369, -21.182405, -19.71780...   
1580  [-17.248318, -16.214254, -15.532733, -15.95249...   
1550  [-24.592573, -24.592529, -24.592484, -25.67959...   

                                                 band_2        id inc_angle  \
797   [-24.019733, -26.013855, -24.019867, -24.24555...  b522fe03   35.6334   
1440  [-24.865326, -26.5518, -27.195549, -30.886093,...  9fabab12    39.234   
728   [-23.390581, -24.356712, -22.945129, -23.16507...  f8c7b912   45.3362   
1580  [-21.718636, -24.217484, -24.443178, -25.15756...  5f49ea3b        na   
1550  [-26.594877, -27.987553, -29.64699, -28.373562...  de13af0f        na   

      is_iceberg     order  
797            1  0.001107  
1440           1  0.001367  
728            1  0.001612  
1580           0  0.001933  
1550           0  0.00202

In [5]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [6]:
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((raw_x_1, raw_x_2), axis = 1)
total_x = raw_x_1
print(total_x.shape)

(1604, 5625)


In [7]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 5625)
(1304, 5625)
(1304, 2)


In [8]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.generator = self.slice_random(x, y, batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            return None, None
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [9]:
# test
test_mini_batch = MiniBatch(dev_x, dev_y, 50)


In [10]:
print(test_mini_batch.next_batch())

(array([[-25.211447, -23.585413, -21.033905, ..., -22.782347, -22.042744,
        -21.867332],
       [-31.97308 , -29.474361, -29.050629, ..., -28.652031, -27.541685,
        -30.395088],
       [-23.622217, -21.182369, -21.182405, ..., -19.87192 , -21.185534,
        -19.282896],
       ..., 
       [-18.457067, -18.005821, -16.045731, ..., -18.012941, -16.411083,
        -18.464401],
       [-16.543774, -16.734745, -21.171787, ..., -19.176027, -19.83946 ,
        -19.049356],
       [-20.029573, -20.029573, -24.320618, ..., -26.20401 , -29.431421,
        -25.081745]]), array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
  

In [11]:
with tf.name_scope('reshape'):
    x_in = tf.placeholder(tf.float32)
    y_in = tf.placeholder(tf.float32)
    x = tf.reshape(x_in, [-1,75,75,1])
    y = tf.reshape(y_in, [-1, 2])

In [12]:
# input N*H*W*C
# output N*H*W*F
def conv_layer(input_data, hight, width, channel, feature, pool_window, name):
    with tf.name_scope(name):
        f = tf.get_variable('f'+name, [width,hight,channel,feature], initializer=tf.contrib.layers.xavier_initializer())
#         f = tf.Variable(dtype=tf.float32, 
#                         initial_value=tf.random_normal([width,hight,channel,feature], mean=0, stddev=0.1), 
#                         expected_shape=[width,hight,channel,feature])
#         b = tf.Variable(dtype=tf.float32,
#                         initial_value=tf.random_normal([feature], mean=0.1, stddev=0.1),
#                         expected_shape=[feature])
        c = tf.nn.conv2d(input_data, filter=f, strides=[1,1,1,1], padding='SAME')
        a = tf.nn.relu(c)
        p = tf.nn.pool(a, pool_window, strides=pool_window, pooling_type='MAX', padding='SAME')
    return p

In [13]:
# input N*W
# output N*W1
def fc_layer(input_data, this_feature, next_feature, name, activate=True):
    with tf.name_scope(name):
        if activate:
            return tf.contrib.layers.fully_connected(input_data, next_feature)
        else:
            return tf.contrib.layers.fully_connected(input_data, next_feature, activation_fn=None)
#         w = tf.Variable(dtype=tf.float32,
#                        initial_value=tf.random_normal([this_feature, next_feature], mean=0, stddev=0.1),
#                        expected_shape=[this_feature, next_feature])
# #         b = tf.Variable(dtype=tf.float32,
# #                        initial_value=tf.random_normal([next_feature], mean=0.1, stddev=0.1), 
# #                        expected_shape=[next_feature])
#         b = tf.constant(0.1, shape=[next_feature])
#         a = tf.matmul(input_data, w) + b
#         if activate:
#             return tf.nn.relu(a)
#         else:
#             return a

In [14]:
def cnn():
    l1 = conv_layer(x, 5, 5, 1, 8, [5,5], 'conv_1')
    l2 = conv_layer(l1, 5, 5, 8, 16, [3,3], 'conv_2')
    l3 = fc_layer(tf.reshape(l2, [-1, 5*5*16]), 5*5*16, 128, 'fc_3')
    l4 = fc_layer(l3, 128, 2, 'fc_4', False)
    return l4

In [15]:
y_ = cnn()

In [16]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_))

In [17]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)

In [18]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

In [19]:
# test
minibatch = MiniBatch(train_x, train_y, 50)
x1,y1 = minibatch.next_batch()

In [20]:
# test
print(x1.shape)
print(y1.shape)

(50, 5625)
(50, 2)


In [21]:
# test
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./logs',sess.graph)
    # test_res = accuracy.eval(session=sess, feed_dict={x_in:x1, y_in:y1})
    test_res = sess.run(y_, feed_dict={x_in:x1, y_in:y1})
    writer.close()

In [22]:
# test
print(test_res)

[[ 29.61538506  -3.00328088]
 [ 30.11032295  -6.09710741]
 [ 41.85795212  -1.74087083]
 [ 40.64450455  -4.68768644]
 [ 28.70099068  -1.87916088]
 [ 36.81660843  -2.91621351]
 [ 36.3024826   -3.18376279]
 [ 40.56246948  -3.41964722]
 [ 36.04766083  -2.75838375]
 [ 41.04793549  -2.76167083]
 [ 28.26268196  -1.45644593]
 [ 28.06295967  -1.13944912]
 [ 34.24255371  -1.97839367]
 [ 28.1107235   -1.79178309]
 [ 40.69409943  -4.1798625 ]
 [ 43.47127533  -3.58862329]
 [ 33.10240936  -4.56947756]
 [ 23.6445713   -2.69055724]
 [ 44.20451736  -2.6673007 ]
 [ 42.71046448  -2.7934382 ]
 [ 37.28145981  -2.51564145]
 [ 31.16433716  -2.46071815]
 [ 28.73022652  -2.77424002]
 [ 37.85688019  -2.76080465]
 [ 43.85177994  -2.87974834]
 [ 35.75405121  -2.28774166]
 [ 37.9142189   -3.10622406]
 [ 33.3874855   -2.6138773 ]
 [ 27.96331978  -2.26697993]
 [ 32.61206055  -1.06613028]
 [ 35.10255432  -2.88066101]
 [ 34.39094543  -4.04317617]
 [ 34.51609039  -3.31586123]
 [ 35.5759964   -2.44716358]
 [ 29.86963081

In [23]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     tf.add_to_collection('accuracy', accuracy)
#     tf.add_to_collection('y_', y_)
#     tf.add_to_collection('x_in', x_in)
#     tf.add_to_collection('y_in', y_in)
#     # writer = tf.summary.FileWriter('./logs',sess.graph)
#     for i in range(300):
#         minibatch = MiniBatch(train_x, train_y, 50)
#         x_batch,y_batch = minibatch.next_batch()
#         if x_batch is None or x_batch.shape[0]<30:
#             minibatch = MiniBatch(train_x, train_y, 50)
#             x_batch,y_batch = minibatch.next_batch()
#         print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
# #         if i%10 == 0:
# #             train_accuracy = accuracy.eval(session=sess,
# #                                             feed_dict={
# #                                                 x_in:x_batch, y_in:y_batch
# #                                             })
# #             print('step {}, training accuracy {}'.format(i, train_accuracy))
# #         sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
# #     # writer.close()
# #     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
# #     saver = tf.train.Saver()
# #     saved_path = saver.save(sess, './models/1st.ckpt')
# #     print('model saved to {}'.format(saved_path))

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('y_', y_)
    tf.add_to_collection('x_in', x_in)
    tf.add_to_collection('y_in', y_in)
    # writer = tf.summary.FileWriter('./logs',sess.graph)
    for i in range(500):
        minibatch = MiniBatch(train_x, train_y, 50)
        x_batch,y_batch = minibatch.next_batch()
        if x_batch is None or x_batch.shape[0]<30:
            minibatch = MiniBatch(train_x, train_y, 50)
            x_batch,y_batch = minibatch.next_batch()
        # print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
        if i%100 == 0:
            train_accuracy = accuracy.eval(session=sess,
                                            feed_dict={
                                                x_in:x_batch, y_in:y_batch
                                            })
            print('step {}, training accuracy {}'.format(i, train_accuracy))
        sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
    # writer.close()
    print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    saver = tf.train.Saver()
    saved_path = saver.save(sess, './models/1channel.ckpt')
    print('model saved to {}'.format(saved_path))

step 0, training accuracy 0.5
step 100, training accuracy 0.8999999761581421
step 200, training accuracy 0.9599999785423279
step 300, training accuracy 0.9800000190734863
step 400, training accuracy 1.0
test accuracy 0.6433333158493042
model saved to ./models/1channel.ckpt


In [25]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('./models/1st.ckpt.meta')
#     saver.restore(sess, './models/1st.ckpt')
#     accuracy = tf.get_collection('accuracy')[0]
#     print('model restored')
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in:mnist.test.images, y_in:mnist.test.labels})))